In [1]:
import os

# Dask Configuration

In [2]:
from dask_jobqueue import PBSCluster
from pathlib import Path

# Define the working directory path
working_directory = str(Path.cwd())

# Launch a scheduler and workers on HPC via PBS
cluster = PBSCluster(
     cores=4,
     memory="8GB",
     processes=1,
     queue="tamirQ",
     walltime="02:30:00",
     scheduler_options={"dashboard_address": ":12435"},  # FIXME - this is the port we need to define...
     # Additional custom options
     log_directory="dask-logs",
     #worker_extra_args=["--lifetime", "25m", "--lifetime-stagger", "4m"],  # for walltime="00:30:00"
     job_script_prologue=[f"cd {working_directory}"]
)

In [3]:
cluster

Dashboard: http://132.66.112.146:12435/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://132.66.112.146:40465,Workers: 0
Dashboard: http://132.66.112.146:12435/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
cluster.adapt(minimum=30, maximum=60)
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -q tamirQ
#PBS -l select=1:ncpus=4:mem=7630MB
#PBS -l walltime=02:30:00
#PBS -e dask-logs/
#PBS -o dask-logs/
cd /tamir2/moranb/microbiome/Igem_TAU_2021
/tamir2/moranb/microbiome/Igem_TAU_2021/venv/bin/python -m distributed.cli.dask_worker tcp://132.66.112.146:40465 --nthreads 4 --memory-limit 7.45GiB --name dummy-name --nanny --death-timeout 60



In [5]:
from dask.distributed import Client, progress, wait, get_client, get_worker
client = Client(cluster)

In [6]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://132.66.112.146:12435/status,
Dashboard: http://132.66.112.146:12435/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://132.66.112.146:40465,Workers: 0
Dashboard: http://132.66.112.146:12435/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
import dask.bag as db
import dask.dataframe as dd
from dask import delayed, compute, persist
import json
from collections import defaultdict
import matplotlib
import numpy as np
import pandas as pd
import re


# Analysis for homo sapiens genome

In [8]:
from Bio import SeqIO
from analysis.orf_model_analysis.input_testing_data.generate_input_testing_data_for_modules import generate_testing_data
from analysis.orf_model_analysis.input_testing_data.generate_input_testing_data_for_modules import generate_testing_data_for_ecoli_and_bacillus

from modules.main import run_modules

In [9]:
output_path = "/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/results/homo_sapiens"

In [10]:
def convert_to_json_result(x):
    gene_name = x[0]
    result = run_modules(x[1], should_run_output_module=False)
    return {
        "initial_optimization_score": result.get("initial_optimization_score"),
        "final_optimization_score": result.get("final_optimization_score"),
        "average_distance_score": result.get("average_distance_score"),
        "weakest_link_score": result.get("weakest_link_score"),
        "gene_name": gene_name,
    }

In [11]:
def extract_ncbi_sequences_for_analysis(fasta_file_path: str) -> None:
    with open(fasta_file_path, "r") as fasta_handle:
        genome_dict = SeqIO.to_dict(SeqIO.parse(fasta_handle, "fasta"))

    missing_genes = []
    gene_mapping = defaultdict(list)
    description_to_gene_mapping = {}

    for record, value in genome_dict.items():
        if "NC" not in record or "XP" in record:
            continue
        parameters = re.findall("gene=.*]", value.description)
        if not parameters:
            missing_genes.append(record)
            continue
        gene_parameter = parameters[0].split("]")[0]
        gene_name = gene_parameter.strip("gene=")

        if len(value.seq) % 3 != 0:
            print(F"Skip sequence {record} for {gene_name} because of length not divisible by 3.")
            continue

        gene_mapping[gene_name].append(record)
        description_to_gene_mapping[record] = gene_name

    with open("gene_mapping.json", "w") as genes_file:
        json.dump(gene_mapping, genes_file)
    with open("description_to_gene_mapping.json", "w") as description_to_genes_file:
        json.dump(description_to_gene_mapping, description_to_genes_file)
    with open("missing_genes.txt", "w") as missing_genes_file:
        for gene in missing_genes:
            missing_genes_file.write(gene)
            
    gene_to_longest_sequence = {
        key: max(value, key=lambda x: len(genome_dict[x].seq)) for key, value in gene_mapping.items()
    }
    with open("gene_to_longest_sequence.json", "w") as genes_file:
        json.dump(gene_to_longest_sequence, genes_file)

In [12]:
homo_sapiens_fasta = "/tamir2/moranb/microbiome/homo_sapiens_genome/cds_from_genomic.fna"
selected_homo_sapiens_genes = "gene_to_longest_sequence.json"

In [13]:
# extract_ncbi_sequences_for_analysis(homo_sapiens_fasta)

## Paenibacillus-prosopidis

In [14]:
# # organism = "Paenibacillus-aceris"
# organism = "Terrabacter-tumescens"


### Two wanted hosts and two unwanted hosts

In [15]:
# optimization_cub_index = "CAI"

# wanted_hosts = [
#     "Paenibacillus-aceris.gb", 
#     "Arthrobacter-luteolus.gb",
# ]
# unwanted_hosts = [
#     "Terrabacter-tumescens.gb", 
#     "Rhodanobacter-denitrificans.gb",
# ]

# configuration = f"wanted_{'_'.join([host[:-3] for host in wanted_hosts])}_unwanted_{'_'.join([host[:-3] for host in unwanted_hosts])}"
# configuration_output_path = os.path.join(output_path, configuration, organism)

In [16]:
# inputs = [(gene_name, generate_testing_data(
#     optimization_method=optimization_method,
#     optimization_cub_index=optimization_cub_index,
#     wanted_hosts=wanted_hosts,
#     unwanted_hosts=unwanted_hosts,
#     tuning_param=0.5,
#     sequence=str(orf_sequence.seq),
#     output_path=os.path.join(configuration_output_path,gene_name))) for gene_name, orf_sequence in genome_dict.items()]

In [17]:
# inputs_series = pd.Series(inputs)
# inputs_db = db.from_sequence(inputs)

An example for running the code on some inputs and saving the results to a file:

In [18]:
# %%time
# partial_inputs = inputs_db.take(10)
# results_db = db.from_sequence(partial_inputs).map(lambda x: {x[0]: run_modules(x[1], should_run_output_module=False)})
# results_db.map(json.dumps).to_textfiles(os.path.join(configuration_output_path, 'results/*.json')) 

An example for reading and plotting the results:

In [19]:
# df = dd.read_json([F"{configuration_output_path}/results/{i}.json" for i in range(3)], orient='index')
# df[["weakest_link_score", "average_distance_score"]].compute().hist()

In [20]:
# %%time
# results_db = inputs_db.map(convert_to_json_result)
# results_db.map(json.dumps).to_textfiles(os.path.join(configuration_output_path, 'results/*.json')) 

In [21]:
# results_dir = F"{configuration_output_path}/zscore_bulk_aa_diff/"
# df = dd.read_json([F"{results_dir}/{file}" for file in os.listdir(results_dir)], orient='index')
# df[["weakest_link_score", "average_distance_score"]].compute().hist()

## Bacillus and E.coli

In [ ]:
with open(homo_sapiens_fasta, "r") as fasta_handle:
    genome_dict = SeqIO.to_dict(SeqIO.parse(fasta_handle, "fasta"))

with open(selected_homo_sapiens_genes, "r") as selected_genes_file:
    selected_genes = json.load(selected_genes_file)

optimization_cub_index = "CAI"

for optimization_method in ["single_codon_ratio",
                            "single_codon_diff", 
                            "zscore_bulk_aa_ratio", 
                            "zscore_bulk_aa_diff"]:
    for is_ecoli_optimized in [True, False]:
        configuration = f"e_coli_optimized_{is_ecoli_optimized}_bacillus_optimized_{not is_ecoli_optimized}"
        configuration_output_path = os.path.join(output_path, configuration)
        
        inputs = [(gene_name, generate_testing_data_for_ecoli_and_bacillus(
            optimization_method=optimization_method,
            optimization_cub_index=optimization_cub_index,
            is_ecoli_optimized=is_ecoli_optimized,
            tuning_param=0.5,
            sequence=str(genome_dict[gene_name].seq),
            output_path=os.path.join(configuration_output_path, gene_name))) for gene_name in selected_genes.values()]
        
        inputs_series = pd.Series(inputs)
        inputs_db = db.from_sequence(inputs)
        
        results_db = inputs_db.map(convert_to_json_result)
        results_db.map(json.dumps).to_textfiles(os.path.join(configuration_output_path, 'results/*.json'))
        results_dir = F"{configuration_output_path}/{optimization_method}/"
        os.rename(F"{configuration_output_path}/results", results_dir)

/tamir2/moranb/microbiome/Igem_TAU_2021/venv/lib/python3.9/site-packages/distributed/client.py:3125: UserWarning: Sending large graph of size 47.18 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initializing it to select=1:ncpus=4:mem=7630MB
Resource specification for PBS not set, initia

In [ ]:
# For testing on partial inputs set

# partial_inputs = inputs_db.take(3)
# results_db = db.from_sequence(partial_inputs).map(convert_to_json_result).compute()

In [ ]:
# df = dd.read_json([F"{results_dir}/{file}" for file in os.listdir(results_dir)], orient='records', lines=True)